In [1]:
import tensorflow as tf
import keras
print(keras.__version__)


2.15.0


In [9]:
import tensorflow as tf
import keras.models
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps  
import numpy as np
import cv2 
import cvzone
import time 

In [12]:
"""  This sub-code is for testing the code """

prob_eff = 0.7
# Load the model and labels 
model = load_model("Model/Model.h5", compile=False)
class_names = open("Model/labels.txt", "r").readlines()

image = cv2.imread('jus_bottle_test.jpg')

resize = tf.image.resize(image, (256, 256))
np.expand_dims(resize, 0).shape
yhat = model.predict(np.expand_dims(resize/255, 0))
print("yhat=", yhat)

# identify the predicted class which cooreponds to the highest predicted probability
predicted_class = np.argmax(yhat)

# Get the corresponding probability
predicted_probability = yhat[0, predicted_class]

print("Predicted Class: ", class_names[predicted_class])
print("Predicted Probability: ", predicted_probability)

1/1 [==============================] - 0s 73ms/step
yhat= [[0.00657227 0.88281345 0.01499238 0.09562192]]
Predicted Class:  1 jus_bottles

Predicted Probability:  0.88281345


In [13]:
# define probability threshold 
prob_eff = 0.7

# Load the trained model and the corresponing labels 
model = load_model("Model/Model.h5", compile=False)
class_names = open("Model/labels.txt", "r").readlines()

#launch camera 
cap = cv2.VideoCapture(0)

continu = True

while continu:
    ret, image = cap.read()  # Capture frame-by-frame 
    if not ret:
        print("End of video")
        break
    
    # resize image to meet prediction model settings  
    resize = tf.image.resize(image, (256, 256))
    np.expand_dims(resize, 0).shape
    
    # launch prediction 
    yhat = model.predict(np.expand_dims(resize/255, 0))

    # identify the predicted class
    index = np.argmax(yhat)

    # Get the corresponding confidence probability
    confidence_score = yhat[0, predicted_class]

    # Define the new frame size for ploting the image 
    new_width, new_height = 600, 400

    # Resize the image to fit the new frame size
    image = cv2.resize(image, (new_width, new_height))

    # Create a blank canvas with the new frame size
    canvas = np.zeros((new_height, new_width, 3), dtype=np.uint8)

    # Place the resized image onto the canvas
    canvas[:image.shape[0], :image.shape[1]] = image

    if index != 0:
        #--------------------------------------------------------------------------------------------
        # In case the machine is not empty : pause execution of program to make adequat action 
        # add delay in the execution of the program
        delay = 5 # delay of 5 seconds 
        time.sleep(delay)
        #--------------------------------------------------------------------------------------------

        if confidence_score > prob_eff: 
            """ if program recognizes bottle """
            result_text = f"Class: {class_names[index]}"
            
            # For testing -----------------------------------------------------------------------------------------
            # display result in video 
            # Get the size of the text
            text_size = cv2.getTextSize(result_text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]

            # Calculate the position to center the text
            text_x = (image.shape[1] - text_size[0]) // 2
            text_y = 30

            # Display the result
            cv2.putText(canvas, result_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            #-------------------------------------------------------------------------------------------------------

            user_input = input("continu? (press 1 for Yes/ 0 for No)")

            try:
                res = int(user_input)
                
            except ValueError:
                print("Invalid input. Please enter 1 or 0.")

            if res == 0:
                continu = False
            
        else:
            """ if program doesn't recognize bottle """
            print("Unkown object")
    
    # Display the frame
    cv2.imshow("Video with Result", canvas)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 32ms/step
